System zarządzania literaturą naukową. Ma on charakter kołowy, pobieraj, analizuj, pobieraj. 
Zaczynamy od pobierania literatury.
Do reasearch warto użyć przeglądarkę firefox + Break Down the Walls. Ustawienie Anna's Library.
Następnie wyciągnięcie zawartości oraz bibliografi (działa lepiej dla artykułów, niż książek)
Strona http://cermine.ceon.pl/index.html - pojedyncze pozycje
Lub aplikacja https://github.com/CeON/CERMINE - odpalana z lini komend, lub jako masowa obróbka plików. 
Ostatnia wersja programu z 2017 roku.
$ java -cp cermine-impl-1.13-jar-with-dependencies.jar pl.edu.icm.cermine.ContentExtractor -output jats -path path/to/directory/with/pdfs/ 
Bibtex bezpośrednio nie działa

In [ ]:
#skrypt do automatycznego wyciągania informacji blibliograficznejz plików pdf.
#!/bin/bash

# Ścieżka do programu java
JAVA_PATH="/usr/bin/java"

# Ścieżka do jar zależności
CERMINE_JAR_PATH="/home/mariusz/cermine-impl-1.13-jar-with-dependencies.jar"

# Ścieżka głównego katalogu do przeszukania
MAIN_DIR="/mnt/backupall/ksiazki_1/"

# Funkcja do sprawdzania czy plik jest mniejszy niż 10MB
function is_smaller_than_10mb {
    local file_size=$(stat -c %s "$1")
    if [[ $file_size -lt 10000000 ]]; then
        return 0
    else
        return 1
    fi
}

# Funkcja do przetwarzania plików PDF
function process_pdf {
    local pdf_dir=$(dirname "$1")
    echo "Przetwarzanie katalogu: $pdf_dir"
    $JAVA_PATH -cp $CERMINE_JAR_PATH pl.edu.icm.cermine.ContentExtractor -outputs jats -path "$pdf_dir"
}

# Główna pętla
while true; do
    # Znajdź wszystkie pliki PDF w podkatalogach
    find "$MAIN_DIR" -type f -name "*.pdf" | while read -r pdf_file; do
        if is_smaller_than_10mb "$pdf_file"; then
            process_pdf "$pdf_file"
        else
            echo "Plik $pdf_file jest większy niż 10MB, pomijanie..."
        fi
    done
    
    # Sprawdź, czy są jeszcze jakieś pliki do przetworzenia
    if [[ $(find "$MAIN_DIR" -type f -name "*.pdf" | wc -l) -eq 0 ]]; then
        echo "Brak plików PDF do przetworzenia, zakończono."
        break
    else
        echo "Znaleziono więcej plików PDF do przetworzenia, kontynuowanie..."
    fi
done

Efektem działania jest katalog z plikami .cermxml Mają one strukturę xml dla literatury naukowej https://jbats.nlm.nih.gov/taglib.html

Kolejny krok to wyciągnięcie danych z plików i zapis ich w pliku csv. Kod w pythonie

In [ ]:
import os
import xmltodict
import csv

# Katalog z plikami CERMXML
directory = "crxml"

# Ścieżka do pliku CSV
csv_file = "bibliography_xml.csv"

# Otwieramy plik CSV w trybie zapisu
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    # Definiujemy nagłówki kolumn
    fieldnames = ["Authors", "Title", "Year", "Journal", "DOI"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    
    # Zapisujemy nagłówki do pliku CSV
    writer.writeheader()

    # Pętla po plikach w katalogu
    for filename in os.listdir(directory):
        if filename.endswith(".cermxml"):
            filepath = os.path.join(directory, filename)
            try:
                # Wczytanie pliku CERMXML
                with open(filepath, "r") as xml_file:
                    xml_data = xml_file.read()

                # Konwersja XML do słownika
                data = xmltodict.parse(xml_data)

                # Sprawdzenie czy istnieje sekcja ref-list
                if "article" in data and "back" in data["article"] and "ref-list" in data["article"]["back"]:
                    # Przetwarzanie tylko elementów ref
                    for reference in data["article"]["back"]["ref-list"].get("ref", []):
                        # Sprawdzenie czy element mixed-citation istnieje
                        mixed_citation = reference.get("mixed-citation", {})
                        if isinstance(mixed_citation, dict):
                            # Pobranie autorów
                            authors = []
                            author_data = mixed_citation.get("string-name", [])
                            if isinstance(author_data, list):
                                for name in author_data:
                                    surname = name.get("surname", "")
                                    given_names = name.get("given-names", "")
                                    full_name = f"{given_names} {surname}"
                                    authors.append(full_name)
                            # Pobranie tytułu
                            title = mixed_citation.get("article-title", "")
                            # Pobranie roku wydania
                            year = mixed_citation.get("year", "")
                            # Pobranie czasopisma
                            journal = mixed_citation.get("source", "")
                            # Pobranie DOI
                            doi = mixed_citation.get("xref", "")

                            # Zapisujemy dane do pliku CSV
                            writer.writerow({
                                "Authors": "; ".join(authors),
                                "Title": title,
                                "Year": year,
                                "Journal": journal,
                                "DOI": doi
                            })

            except Exception as e:
                # Obsługa błędów: zapisz informacje o błędzie do pliku dziennika
                with open("error_log.txt", "a") as log_file:
                    log_file.write(f"Error processing file {filepath}: {str(e)}\n")
                continue  # Kontynuuj przetwarzanie kolejnych plików

Następnie można zrobić Name Entity Recognition (NER) aby rozpoznać nazwiska z kolumy Authors.

In [ ]:
import os
import spacy
from langdetect import detect
import pandas as pd
from tqdm import tqdm

# Ścieżka do folderu zawierającego pliki cermxml
cermxml_folder = "crxml"

# Funkcja do przeprowadzenia detekcji NER dla danego pliku
def detect_ner(file_path, language):
    if language == "pl":
        model_name = "pl_core_news_lg"
    elif language == "en":
        model_name = "en_core_web_md"
    else:
        raise ValueError("Unsupported language")
    nlp = spacy.load(model_name)
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
        doc = nlp(text)
        entities = [ent.text for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE']]
    return entities

# Lista przechowująca wyniki detekcji
results = []
skipped_files = []

# Przeprowadzenie detekcji NER dla każdego pliku
files = [file for file in os.listdir(cermxml_folder) if file.endswith(".cermxml")]
for file in tqdm(files, desc="Processing files", unit="file"):
    file_path = os.path.join(cermxml_folder, file)
    # Wykrycie języka dla danego pliku
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
        try:
            language = detect(text)
        except:
            language = "unknown"
    try:
        entities = detect_ner(file_path, language)
        results.append({"source": file, "entities": entities})
    except ValueError as e:
        skipped_files.append(file)
        print(f"Skipping file {file}: {e}")

# Utworzenie ramki danych
df = pd.DataFrame(results)

# Zapisanie ramki danych do pliku CSV
df.to_csv("cermxml-NER.csv", index=False)

# Zapisanie informacji o pominiętych plikach do pliku tekstowego
with open("skipped_files.txt", "w") as f:
    f.write("\n".join(skipped_files))


Czysczenie plików. Krok pierwszy - usuwanie pustych linijek. Pliki cermxml czasem nie są właściwie ustrukturyzowane.

In [ ]:
import pandas as pd

# Wczytaj plik CSV
df = pd.read_csv("cermxml-NER.csv")

# Usuń puste wiersze
df = df.dropna()

# Funkcja do przekształcenia listy encji w czytelną postać
def format_entities(entities):
    # Łączy elementy listy w pojedynczy ciąg znaków, oddzielając je przecinkiem
    return ', '.join(entities)

# Przekształć listy encji w czytelną postać
df['entities'] = df['entities'].apply(eval).apply(format_entities)

# Zapisz oczyszczoną ramkę danych do pliku CSV
df.to_csv("cermxml-NER-cleaned.csv", index=False)

Usuwanie znaczników xml

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

# Wczytaj plik CSV
df = pd.read_csv("cermxml-NER-cleaned.csv")

# Usuń puste wiersze
df = df.dropna()

# Funkcja do parsowania listy encji
def parse_entities(entities_str):
    # Podziel listę encji na pojedyncze elementy
    entities_list = entities_str.strip("[]").split(", ")
    # Usuń cudzysłowy i znaki specjalne z każdego elementu listy
    cleaned_entities = [entity.strip("''").replace("¨", "") for entity in entities_list]
    return cleaned_entities

# Parsuj listy encji i usuń znaczniki HTML
df['entities'] = df['entities'].apply(parse_entities)
df['entities'] = df['entities'].apply(lambda x: ', '.join(x))
df['entities'] = df['entities'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())

# Zapisz oczyszczoną ramkę danych do pliku CSV
df.to_csv("cermxml-NER-cleaned-html.csv", index=False)

#Kolejna część to analiza tematów. Klasyfikacja odbywa się przy użyciu BERTa i SpacY.

In [ ]:
import pandas as pd
from bertopic import BERTopic
from tqdm import tqdm

try:
    # Wczytaj plik CSV
    df = pd.read_csv("bibliography_xml.csv")

    # Usuń puste wartości w kolumnie 'Title'
    df = df.dropna(subset=['Title'])

    # Wybierz tylko kolumnę 'Title'
    titles = df['Title'].tolist()

    # Inicjalizuj BERTopic
    topic_model = BERTopic()

    # Dopasuj model do danych
    try:
        with tqdm(total=len(titles), desc="Fitting BERTopic") as pbar:
            topic_model.fit(titles)
            pbar.update(1)
    except Exception as e:
        print("An error occurred during model fitting:", e)

    # Przypisz etykiety tematów z powrotem do ramki danych
    try:
        topic_labels, _ = topic_model.transform(titles)
        df['Topic'] = topic_labels
    except Exception as e:
        print("An error occurred while assigning topic labels:", e)

    # Mapuj identyfikatory tematów na ich nazwy
    topic_freq = topic_model.get_topic_freq()
    topic_name_map = {topic[0]: f"Topic {topic[0]}" for topic in topic_freq}

    # Dodaj kolumnę 'Topic Name' z nazwami tematów
    df['Topic Name'] = df['Topic'].map(topic_name_map)

    # Zapisz wyniki do pliku CSV
    df[['Title', 'Topic', 'Topic Name']].to_csv("wyniki_bertopic.csv", index=False)

    # Wyświetl tylko 10 pierwszych wierszy
    print(df[['Title', 'Topic', 'Topic Name']].head(10))

except Exception as e:
    print("An error occurred:", e)

In [ ]:
# Pobierz słownik etykiet tematów i odpowiadających im słów kluczowych
topics = topic_model.get_topics()

# Przejrzyj słownik etykiet tematów i przypisanych im słów kluczowych
for topic, keywords in topics.items():
    print(f"Topic {topic}: {keywords}")

Stworzenie listy najpopilarniejszych słów (rzeczowników i czasowników) w formach podstawowych. Kod optymalizowany, żeby działał na wielu rdzeniach. 

In [ ]:
import os
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
from langdetect import detect
from tqdm import tqdm
import spacy
from concurrent.futures import ThreadPoolExecutor

# Load the Polish and English language models
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_sm")

# Define a function to clean XML tags from text
def clean_xml_tags(text):
    soup = BeautifulSoup(text, "xml")
    return soup.get_text()

# Define a function to extract lemmas from text
def extract_lemmas(text, language):
    if language == 'pl':
        nlp = nlp_pl
    elif language == 'en':
        nlp = nlp_en
    else:
        return None
    # Tokenize the text
    doc = nlp(text)
    # Initialize counters for verbs and nouns
    verb_counter = Counter()
    noun_counter = Counter()
    # Count verbs and nouns
    for token in doc:
        if token.pos_ == 'VERB':
            verb_counter[token.lemma_] += 1
        elif token.pos_ == 'NOUN':
            noun_counter[token.lemma_] += 1
    # Filter lemmas to include only verbs and nouns occurring more than 30 times
    filtered_lemmas = {lemma: count for lemma, count in verb_counter.items() if count > 15}
    filtered_lemmas.update({lemma: count for lemma, count in noun_counter.items() if count > 15})
    return filtered_lemmas

# Define a function to process a single file
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        # Clean XML tags
        cleaned_text = clean_xml_tags(text)
        try:
            language = detect(cleaned_text)
            lemmas = extract_lemmas(cleaned_text, language)
            if lemmas:
                return lemmas
        except:
            return None

# Define the directory containing the XML files
directory = 'crxml'

# Initialize list to store results
results = []

# Process each file in the directory using concurrent processing
with ThreadPoolExecutor() as executor:
    futures = []
    for filename in os.listdir(directory):
        if filename.endswith('.cermxml'):
            file_path = os.path.join(directory, filename)
            future = executor.submit(process_file, file_path)
            futures.append((filename, future))

    # Retrieve results from futures
    for filename, future in tqdm(futures):
        lemmas = future.result()
        if lemmas:
            for lemma, count in lemmas.items():
                results.append((filename, lemma, count))

# Create a DataFrame from the results
df = pd.DataFrame(results, columns=["nazwa_pliku", "lemma", "liczba"])

# Save the DataFrame to a CSV file
df.to_csv("wyniki_analizy-opt.csv", index=False)

# Print the DataFrame
print(df)